# Get Playlists items from Spotify using spotipy library

In [22]:
import os,json
from flask import Flask, redirect,session, url_for, request
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from spotipy.cache_handler import FlaskSessionCacheHandler

In [23]:
client_id = "258ba1badd9247c9b88999500e883d59"
client_secret = "d45f2362c5bd42718060bd8a58038f32"
redirect_uri = 'http://localhost:5000/callback'
scope= 'playlist-read-private' 
cache_handler = FlaskSessionCacheHandler(session)

In [24]:
sp = Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=scope,
    open_browser=True  # Opens a browser for authentication
))


In [26]:
def get_playlists():
    """Fetch the current user's playlists."""
    playlists = []
    results = sp.current_user_playlists()

    while results:
        for item in results["items"]:
            playlists.append({
                "id": item["id"],
                "name": item["name"],
                "url": item["external_urls"]["spotify"]
            })
        results = sp.next(results) if results["next"] else None

    return playlists


In [27]:
# Fetch and save playlists
playlists_data = get_playlists()

with open("playlists.json", "w", encoding="utf-8") as f:
    json.dump({"items": playlists_data}, f, indent=4, ensure_ascii=False)

print("✅ Playlists saved to playlists.json")


✅ Playlists saved to playlists.json


### Getting Playlist items i.e. Tracks

In [28]:
# Load playlists from JSON file
with open("playlists.json", "r", encoding="utf-8") as f:
    playlists = json.load(f)

In [35]:
# Loop through playlists and fetch tracks
tracks_data = {}
for playlist in playlists.get("items", []):
    playlist_id = playlist.get("id")
    playlist_name = playlist.get("name", "Unknown Playlist")
    print(f"Fetching tracks for {playlist_name} ({playlist_id})...")
    tracks_data[playlist_name] = sp.playlist_items(playlist_id)

Fetching tracks for Oldies (7sNycTeMl6JLt9s3BFL2ld)...
Fetching tracks for Sad Hindi Songs 😔💔  (Mood Off 🥺 )  (189Sow1xr7R94oSKs4kISc)...
Fetching tracks for Ghazal Playlist (0iF2mH8FoVaM9Bz8CEK9T3)...
Fetching tracks for rafta rafta woh meri🌙✨ (69BKWs1DWxZFajArBy46Fm)...
Fetching tracks for Romantic Mix (2) (765QDF4kIYQFjM5d9PHgYS)...
Fetching tracks for Gym (5GMx2htp7H6K9nzCG3LKx2)...
Fetching tracks for Gala (32Zl4adRA20vp2evFLFBMc)...
Fetching tracks for Deep Focus (19btKKOB255dgQsBhN4knw)...
Fetching tracks for Tera deedar hua,pehla sa pyaar hua ❤️ (0VO64r49n6jThxu1f4OEY2)...
Fetching tracks for Tunes 2k18 to 2k19 (74Lq9FhBnCiqVBipYHhBk7)...
Fetching tracks for Pakistani Drama Songs New (1XECRXuaUsjdnvfls5jNK1)...
Fetching tracks for Gi (0kIWuUTcXSbr3VG1VzDVSf)...
Fetching tracks for Chores (4QYiIqvMYd0ztwkMiUIqR0)...
Fetching tracks for  (1XhyuQjJaAoEuYueolk018)...
Fetching tracks for 8D Music 2024 🎧 8D Audio | 8D Songs by 8D Tunes (2ULLEdMWHTYLEhNbaclKwS)...
Fetching tracks for 

In [36]:
# Save to tracks.json
with open("tracks.json", "w", encoding="utf-8") as f:
    json.dump(tracks_data, f, indent=4, ensure_ascii=False)

print("Tracks saved to tracks.json")

Tracks saved to tracks.json


In [2]:
%pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import json 
import pandas as pd

In [8]:
with open("tracks.json", "r" , encoding= 'utf-8') as file:
    data = json.load(file)

In [26]:
tracks = []
for playlist_name, playlist_data in data.items():
    if 'items' in playlist_data:
        for item in playlist_data['items']:
            track = item.get('track', {})
            track_name = track.get('name')
            track_id = track.get('id')
            if track_name and track_id:
                tracks.append({'Track name': track_name, 'Track id': track_id})

In [27]:
df = pd.DataFrame(tracks)

In [28]:
df.to_csv('tracks.csv')

In [29]:
df

,Track name,Track id
0,Itna Na Mujhse Tu Pyar Badha - Duet,5U3LJ3XtA63pGWMeskRULX
1,Aksar Is Duniya Mein,6m2UYr6ZJN8wtDzOauBIb9
2,Teri Baaton,7BgYR9r202eRYKMgCtebII
3,Abhi Na Jao Chhod Kar,2oaKoKA8bvgWfBi9M8wEVJ
4,"Hothon Se Chhu Lo Tum - From ""Prem Geet""",4mBmsPcPa1Eu4LDTHq55Ab
...,...,...
1818,Halka Halka,5IPQRERKcsXeaKlRpEJr28
1819,Tu Jaane Na,4iFPsNzNV7V9KJgcOX7TEO
1820,Allah Waariyan,6YYVzVapUF3UnZQ0igasl3
1821,"Humdard (From ""Ek Villain"")",2PvaburAUlhNHxVhw5lvOq
